In [13]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import scipy as sc
from skimage import transform

In [14]:
from os import listdir
from os.path import isfile, join
path1 = 'data/symptoms_new/'
path2 = 'data/nosymptoms_new/'
file_pos = [path1+f for f in listdir(path1) if isfile(join(path1, f)) and f[-4:] == 'jpeg']
file_neg = [path2+f for f in listdir(path2) if isfile(join(path2, f)) and f[-4:] == 'jpeg']

In [15]:
len(file_pos), len(file_neg)

(595, 1468)

In [24]:
import pandas as pd
file1 = pd.DataFrame()
file1['location'] = file_pos
file1['label'] = 1

file2 = pd.DataFrame()
file2['location'] = file_neg
file2['label'] = 0
file = pd.concat([file1, file2], axis=0)
file = file.sample(frac=1)

In [26]:
file_names = file.location
data = []
for file in file_names:
    img = sc.ndimage.imread(file)
    image = np.dot(img[...,:3], [0.299, 0.587, 0.114])
    data.append(image)
data = np.array(data)

In [32]:
#normalize
data = data/255